In [1]:
from langkit import llm_metrics, extract
import re
import json
import time
import pandas as pd
headers_openllm = {"Content-Type": "application/json", "Accept": "application/json","AUTHORIZATION": "eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJ4WTdTd3k5UE1xaXRDQmNSMm5qcVl6bmoxS3NqZzV3TmdOV0xDVzdyUkhvIn0.eyJleHAiOjE3MzA4ODM3MzgsImlhdCI6MTY5OTI2MTMzOCwiYXV0aF90aW1lIjoxNjk5MjUyMDQ0LCJqdGkiOiJmN2EzMzQwYy1kNDQwLTRlMzUtYjk2ZS04YzBiMTc0Y2RhODAiLCJpc3MiOiJodHRwczovL3JlZnJhY3QtbG9naW4uZm9zZm9yLmNvbS9hdXRoL3JlYWxtcy9tb3NhaWMiLCJhdWQiOlsibW9zYWljLWdhdGVrZWVwZXIiLCJhY2NvdW50Il0sInN1YiI6IjZjMjU4MWU3LWZmMTItNDljNy04MDJmLWI2ZjQzOWQxZDIwMSIsInR5cCI6IkJlYXJlciIsImF6cCI6Im1vc2FpYy1nYXRla2VlcGVyIiwic2Vzc2lvbl9zdGF0ZSI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJNTE9QUyIsImxvbmdfbGl2ZWRfdG9rZW4iLCJzcGVjdHJhLWRldmVsb3BlciIsImRlZmF1bHQtcm9sZXMtbW9zYWljIiwicmVmcmFjdC1kZXZlbG9wZXIiLCJvZmZsaW5lX2FjY2VzcyIsImFkbWluIiwidW1hX2F1dGhvcml6YXRpb24iLCJyZWZyYWN0LWFkbWluIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl19fSwic2NvcGUiOiJvcGVuaWQgZW1haWwgcHJvZmlsZSIsInNpZCI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImVtYWlsX3ZlcmlmaWVkIjp0cnVlLCJuYW1lIjoiUmVmcmFjdCBCRlNJIiwicHJlZmVycmVkX3VzZXJuYW1lIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20iLCJnaXZlbl9uYW1lIjoiUmVmcmFjdCIsImZhbWlseV9uYW1lIjoiQkZTSSIsImVtYWlsIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20ifQ.b6SYLgjo9Veo3GmJ8eZjCTNupQjpfMhzsoXdYjWwRtvRnNjBfx0gOqcugO9OcGn-mm8wwpSGI5uiL30-I6SdWBjsf1ur6GztoX7j-nP_3SrJJn3UhNNqIO8LbsPi5gGRTzWtnfjz92BF1YaCXxQwPY0P_aa8vJ6JxZz5Uctn9aIPIJZZnnjC_GPXtXurmshM_tEN2kwCjhEyr7wYzRqUoMtBGfpLjZREBzgZY-x6JyYiXNtycb1d6PFcCXf7nJVV8ienEC_x7OuciDzfeqd-SQnImvAHH7rqFdi9smBN08AbkDS2uAbMrokHrmbiBpaimrR013VwCWz2KL5QYlWleA"}
url_openllm = 'https://refract.fosfor.com/vllm/mistral/v1/completions'

[nltk_data] Downloading package vader_lexicon to /home/mosaic-
[nltk_data]     ai/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
/tmp/pip_packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


[2024-03-06 09:36:03,378] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
def narrative_generation_with_openllm(intermediate_result, question):
  prompt = """Given the input :\n"""+str(intermediate_result)+"""\nAnd the query: """+ str(question)+"""\n is converted into below narrative.\n[Narrative]"""
  data_openllm = {"prompt": prompt,
                  "max_tokens": 200,
                  "temperature": 0.01,
                  "model": "/llmmodels/NLG_FINETUNEDMODELS/DATASET_MODELS/FINETUNED_MISTRALV2_V5/MISTRAL_MERGED_MODEL_JAN30_V3_0021",
                  "stop": "["}
  start = time.time()
  response = requests.post(url_openllm, json=data_openllm, headers=headers_openllm)
  end = time.time()
  open_llm_narrative_json = json.loads(response.text)
  open_llm_narrative = re.search(r'\n([^|\n]*)', open_llm_narrative_json["choices"][0]["text"]).group(1)
  open_llm_response_time = str(end - start) + str(" seconds")

  output_dict = {
    "opensource_llm_narrative":open_llm_narrative,
    "opensource_llm_response_time":open_llm_response_time
  }
  print(output_dict)

In [5]:
question = input("Enter the question")
intermediate_result = input("Enter the intermediate_result")
#Prompt validation
data = [[question,""]]
prompt_response = pd.DataFrame(data, columns=['prompt', 'response'])
metrics_df = extract(prompt_response)
print(metrics_df.columns)
metrics_df


#narrative_generation_with_openllm(intermediate_result,question)

Enter the questionmonthly trend of sales across market in 2021
Enter the intermediate_result{'Date_Interval': 'month', 'Latest_period': '2021-12-01', 'Total_Latest_Value': 200609.99, 'Latest_Period_Constant_Measures': False, 'Latest_Top1': ['APAC'], 'Latest_Top1_Values': [53514.51], 'Latest_Top2': ['US'], 'Latest_Top2_Values': [47591.55], 'Latest_Bottom1': ['Canada'], 'Latest_Bottom1_Values': [270.6], 'Negative_present': False, 'Measure': [{'label': 'Sales', 'variable': 'Sales', 'type': 'measure', 'alternate_usage': ['Sales'], 'isRegional': False, 'number_type': 'add_values'}], 'Dimension': [{'label': 'Order Date', 'variable': 'Order Date', 'type': 'dimension', 'alternate_usage': ['Order Date'], 'isRegional': False, 'singular': 'Order Date', 'plural': 'Order Dates'}, {'label': 'Market', 'variable': 'Market', 'type': 'dimension', 'alternate_usage': ['Market'], 'isRegional': False, 'singular': 'Market', 'plural': 'Markets'}], 'Dimension_1_len': 12, 'Dimension_2_len': 7, 'Filter': {'Order

,prompt,response,prompt.has_patterns,response.has_patterns,prompt.sentiment_nltk,response.sentiment_nltk,prompt.flesch_reading_ease,response.flesch_reading_ease,prompt.automated_readability_index,response.automated_readability_index,...,response.polysyllable_count,prompt.monosyllable_count,response.monosyllable_count,prompt.difficult_words,response.difficult_words,prompt.jailbreak_similarity,response.refusal_similarity,prompt.toxicity,response.toxicity,response.relevance_to_prompt
0,monthly trend of sales across market in 2021,,None,None,0.0,0.0,88.74,206.84,4.4,0.0,...,0,6,0,1,0,0.071328,0.207186,0.000754,0.002407,0.126902


In [15]:
metrics_df['prompt.toxicity'].values[0]

0.0007538795471191406